<html><font size=6><B>3η εργαστηριακή άσκηση</B><BR>
    <BR><B>Link prediction - Πρόβλεψη συνδέσμων</B></BR>
    </font></html>

## 1

<html><font size=4><B>1. Κατασκευή γράφου και προεργασία για την πρόβλεψη συνδέσμων</B></font>
    <li><B>1(i).</B> Χρησιμοποιήστε το dataset <a href="http://konect.cc/networks/dbpedia-similar/"> Similarities (DBpedia)</a>, για να κατασκευάσετε τον μη κατευθυνόμενο (undirected) γράφο $G$ <a href="https://dl.acm.org/doi/10.1145/2487788.2488173">[1]</a>,<a href="https://link.springer.com/chapter/10.1007/978-3-540-76298-0_52">[2]</a>. Οι κόμβοι του γράφου αναπαριστούν σελίδες της Wikipedia και οι ακμές σχέση ομοιότητας μεταξύ τους. Στην σελίδα <a href="http://konect.cc/networks/dbpedia-similar/"> KONECT (DBpedia)</a> μπορείτε να βρείτε πληροφορίες για τις ιδιότητες του γράφου.</a><BR>
        <B><U>Υπόδειξη:</U></B> Μπορείτε να ελέγξετε προγραμματιστικά αν το dataset περιέχει διπλότυπα των ακμών με την ίδια ή αντίστροφη σειρά (π.χ. ακμές (a,b) και (b,a)), και να λάβετε υπ' όψιν σας αυτήν σας την παρατήρηση, όπου τυχόν χρειαστεί, στην ερώτηση 2.<BR></html>

### 1(i)

In [1]:
from functools import wraps
from multiprocessing import cpu_count

import numpy as np
import pandas as pd
import networkx as nx
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from node2vec.node2vec import Node2Vec, gensim

RANDOM_STATE = 42
K = int('03400149') % 10


class Node2VecFixed(Node2Vec):

    def fit(self, **skip_gram_params):
        size_kwarg = 'size' if tuple(gensim.__version__.split('.')) < ('4',) else 'vector_size'

        if 'workers' not in skip_gram_params:
            skip_gram_params['workers'] = self.workers

        if size_kwarg not in skip_gram_params:
            skip_gram_params[size_kwarg] = self.dimensions

        return gensim.models.Word2Vec(self.walks, **skip_gram_params)

In [2]:
G: nx.Graph = nx.read_edgelist(
    'dbpedia-similar/out.dbpedia-similar',
    comments='%',
)

In [3]:
G.is_directed()

False

In [4]:
selfloops = set(nx.selfloop_edges(G))
selfloops

{('420', '420')}

In [5]:
G.remove_edges_from(selfloops)

 <li><B>1(ii).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G$.</li><BR>

### 1(ii)

In [6]:
def graph_summary(G: nx.Graph):
    summary = (
        f'Number of nodes: {G.number_of_nodes()}\n'
        f'Number of edges: {G.number_of_edges()}\n'
        f'Average node degree: {np.mean([degree for _, degree in G.degree()])}\n'
        f'Number of connected components: {nx.number_connected_components(G)}\n'
    )
    return summary


print(graph_summary(G))

Number of nodes: 430
Number of edges: 564
Average node degree: 2.6232558139534885
Number of connected components: 95



<li><B>1(iii). </B>Υπολογίστε το σύνολο των ασύνδετων ζευγών κόμβων (unconnected pairs) στον γράφο χρησιμοποιώντας τον πίνακα γειτνίασης (adjacency matrix). Δώστε επίσης τον αριθμό τους. Λάβετε υπ' όψιν σας ότι ο γράφος είναι μη κατευθυνόμενος και δεν χρειάζεται να διατρέξετε ολόκληρο τον πίνακα γειτνίασης.</li><BR>

### 1(iii)

In [7]:
adj = nx.to_scipy_sparse_array(G)
triu_indices = np.triu_indices_from(adj, k=1)
zero_triu_indices_mask = adj[triu_indices] == 0
zero_triu_indices = triu_indices[0][zero_triu_indices_mask], triu_indices[1][zero_triu_indices_mask]
nodes = np.array(list(G.nodes()))
unconnected_pairs = list(zip(nodes[zero_triu_indices[0]], nodes[zero_triu_indices[1]]))

print(f'Number of unconnected pairs: {len(unconnected_pairs)}')

Number of unconnected pairs: 91671


<li><B>1(iv).</B> Υπολογίστε το σύνολο των ακμών (removable edges) που μπορούν να αφαιρεθούν από τον γράφο χωρίς να υπάρχει κατάτμηση (splitting) του γράφου (δηλ. ο αριθμός των συνεκτικών συνιστωσών θα πρέπει να παραμένει αμετάβλητος). Υπόδειξη: Ελέγχετε επίσης ότι με αφαίρεση ακμής δεν μειώνεται ο αριθμός των κόμβων (εφόσον δεν υπάρχει ξεχωριστό αρχείο μόνο με κόμβους στο dataset). Δώστε επίσης τον αριθμό των ακμών (removable edges) που μπορούν να αφαιρεθούν υπό αυτές τις προϋποθέσεις.</li><BR>

### 1(iv)

In [8]:
removable_edges = []
G_reduced = G.copy()

for u, v in G.edges():
    G_reduced.remove_edge(u, v)

    if (
        nx.number_connected_components(G_reduced) == nx.number_connected_components(G) and
        nx.number_of_nodes(G_reduced) == nx.number_of_nodes(G)
    ):
        removable_edges.append((u, v))
    else:
        G_reduced.add_edge(u, v)

len(removable_edges)

229

<li><B>1(vα).</B> Κατασκευάστε dataframe που συνενώνει τις απαντήσεις των ερωτημάτων 1(iii) και 1(iv) (unconnected+removables). Χρησιμοποιήστε την μεταβλητή $link$ που θα παίρνει τιμή 1 ή 0 ανάλογα με την ύπαρξη συνδέσμου ή μη. To dataframe αυτό συγκεντρώνει τα θετικά και αρνητικά δείγματα για την πρόβλεψη των μελλοντικών συνδέσεων (link: target variable). <BR>
    <li><B>1(vβ).</B> Κατασκευάστε τον γράφο $G'$ που προκύπτει από την αφαίρεση από τον γράφο $G$ των ακμών του ερωτήματος 1(iv). <BR>
    <li><B>1(vγ).</B> Εμφανίστε τον αριθμό των κόμβων, τον αριθμό των ακμών, τον μέσο βαθμό των κόμβων και τον αριθμό των συνεκτικών συνιστωσών (connected components) για τον γράφο $G'$.</li><BR>

### 1(vα)

In [9]:
ground_truths = pd.Series(
    data=[0] * len(unconnected_pairs) + [1] * len(removable_edges),
    index=unconnected_pairs + removable_edges,
    name='is_edge'
)

### 1(vβ)

Είναι το G_reduced του 1(iv)

### 1 (vγ)

In [10]:
print('Original Graph', graph_summary(G), 'Reduced Graph', graph_summary(G_reduced), sep='\n')

Original Graph
Number of nodes: 430
Number of edges: 564
Average node degree: 2.6232558139534885
Number of connected components: 95

Reduced Graph
Number of nodes: 430
Number of edges: 335
Average node degree: 1.558139534883721
Number of connected components: 95



## 2

<html><font size=4><B>2. Εισαγωγή σε similarity-based μετρικές για Link prediction</B></font><BR><BR>
     <li> Η μετρική <B>Jaccard Coefficient (JC)</B> των κόμβων ${u}$ και ${v}$ ορίζεται ως $\frac{|\Gamma(u) \cap \Gamma(v)|}{|\Gamma(u) \cup \Gamma(v)|}$, όπου ${\Gamma(u)}$ είναι το σύνολο των γειτονικών κόμβων του κόμβου ${u}$.  <B></B>
        <li> Η μετρική <B>Preferential Αttachment (PA)</B> των κόμβων $u$ και $v$ ορίζεται ως $|\Gamma(u)| |\Gamma(v)|$, όπου $\Gamma(u)$ είναι το σύνολο των γειτονικών κόμβων του $u$.</li>
        <li> Η μετρική <B>Resource Allocation (RA)</B> των κόμβων $u$ και $v$ ορίζεται ως $\sum_{w \in \Gamma(u) \cap \Gamma(v)} \frac{1}{|\Gamma(w)|}$, όπου $\Gamma(u)$ είναι το σύνολο των γειτονικών κόμβων του $u$.</li><BR><BR>
    <li><B>2(i).</B> Για τον γράφο $G'$, που κατασκευάσατε στο ερώτημα 1(vβ), υπολογίστε την μετρική <B>Jaccard Coefficient (JC)</B>.  <u><a href="https://networkx.org/documentation/stable/reference/algorithms/link_prediction.html?highlight=link%20prediction">Χρησιμοποιήστε την υλοποιημένη συνάρτηση του λογισμικού NetworkX.<a></u> <BR><BR>
    <li><B>2(ii). </B>Δώστε τις $k$ ακμές με την υψηλότερη τιμή μετρικής $JC$, όπου $k$ ο $Α.Μ.$ σας $mod$ $10$. Για $Α.Μ.$ λήγοντα σε $0$ δώστε τις $10$ ακμές με την υψηλότερη τιμή μετρικής.
        (Για την σύγκριση μεταξύ διαφορετικών μετρικών προτιμάται η εύρεση των $k$ επικρατέστερων συνδέσμων, αφού οι μετρικές επιστρέφουν διαφορετικό εύρος τιμών.)</li><BR>
    <li><B>2(iii). </B>Προσδιορίστε για την μέτρηση της απόδοσης της μετρικής $JC$ τα μεγέθη $Precision$, $Recall$ και $Accuracy$ και αναπαραστήστε τα γραφικά για τιμές κατωφλίου: $0.1$, $0.15$, $0.25$, $0.5$, $1.0$.  <BR><BR>
    $Precision=\frac{TP}{TP+FP}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $FP$: ψευδώς θετικά αποτελέσματα <BR>
    $Recall=\frac{TP}{TP+FN}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $FN$: ψευδώς αρνητικά αποτελέσματα</li> 
    $Accuracy=\frac{TP+TN}{TP+FN+TN+FP}$, όπου $TP$: αληθώς θετικά αποτελέσματα, $TN$: αληθώς αρνητικά αποτελέσματα, $FN$: ψευδώς αρνητικά αποτελέσματα, $FP$: ψευδώς θετικά αποτελέσματα</li><BR><BR>
        <B><U>Υπόδειξη:</U></B>
        Θετικό θεωρήστε ένα αποτέλεσμα, αν η μετρική $JC$ υπερβαίνει μια τιμή κατωφλίου (>=). Αρνητικό θεωρήστε το στην αντίθετη περίπτωση.
        <li>Αληθώς θετικά είναι τα αποτελέσματα που βγήκαν θετικά (η τιμή της $JC$ υπερβαίνει την τιμή κατωφλίου) και πράγματι θα υπάρξουν οι συνδέσεις που περιγράφουν (ερ. 1(iv)).</li>
        <li>Αληθώς αρνητικά είναι τα αποτελέσματα που βγήκαν αρνητικά και πράγματι δεν θα υπάρξουν οι συνδέσεις που περιγράφουν (ερ. 1(iii)).</li>
        <li>Ψευδώς θετικά είναι τα αποτελέσματα που βγήκαν θετικά, ενώ δεν θα υπάρξουν οι συνδέσεις που περιγράφουν.</li>
        <li>Ψευδώς αρνητικά είναι τα αποτελέσματα που βγήκαν αρνητικά, ενώ θα υπάρξουν οι συνδέσεις που περιγράφουν.</li>
        <li>Το άθροισμα $TN+FP$ πρέπει κάθε φορά να σας δίνει όλες τις ασύνδετες ακμές του γράφου $G$.</li>   
<li>Από το ερώτημα 1(iii) οι ακμές που υπολογίσατε είναι αυτές για τις οποίες δεν θα υπάρξουν συνδέσεις.</li> <li>Από το ερώτημα 1(iv) οι ακμές που υπολογίσατε είναι αυτές για τις οποίες θα υπάρξουν συνδέσεις.</li><BR></html>

### 2(i)

In [11]:
def link_predictor_wrapper(link_predictor):

    @wraps(link_predictor)
    def inner(G, ebunch, **kwargs):
        pairs = []
        coefficients = []

        for u, v, coefficient in link_predictor(G, ebunch, **kwargs):
            pairs.append((u, v))
            coefficients.append(coefficient)

        coefficients_series = pd.Series(data=coefficients, index=pairs, name=link_predictor.__name__)
        coefficients_series = coefficients_series.loc[ebunch]

        return coefficients_series

    return inner


jc = link_predictor_wrapper(nx.jaccard_coefficient)(G_reduced, ground_truths.index)
jc.value_counts()

0.000000    91391
1.000000      247
0.500000       84
0.333333       75
0.250000       42
0.166667       20
0.200000       19
0.142857       10
0.125000        6
0.111111        2
0.090909        1
0.083333        1
0.071429        1
0.076923        1
Name: jaccard_coefficient, dtype: int64

### 2 (ii)

In [12]:
jc.nlargest(K)

(4, 46)      1.0
(4, 47)      1.0
(4, 49)      1.0
(46, 47)     1.0
(46, 49)     1.0
(47, 49)     1.0
(23, 34)     1.0
(27, 33)     1.0
(100, 92)    1.0
Name: jaccard_coefficient, dtype: float64

### 2 (iii)

In [13]:
link_predictors = nx.jaccard_coefficient, nx.preferential_attachment, nx.resource_allocation_index
thresholds = 0.1, 0.15, 0.25, 0.5, 1.0
metrics = precision_score, recall_score, accuracy_score

res = []

for link_predictor in map(link_predictor_wrapper, link_predictors):
    vals = link_predictor(G_reduced, ground_truths.index)
    for metric in metrics:
        for threshold in thresholds:
            predictions = (vals >= threshold).astype(int)
            score = metric(predictions, ground_truths)
            res.append((link_predictor.__name__, metric.__name__, threshold, score))

pd.DataFrame(res, columns=['link_predictor', 'metric', 'threshold', 'score'])

/home/konstantinos/miniconda3/envs/sna/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,link_predictor,metric,threshold,score
0,jaccard_coefficient,precision_score,0.10,0.707424
1,jaccard_coefficient,precision_score,0.15,0.689956
2,jaccard_coefficient,precision_score,0.25,0.676856
3,jaccard_coefficient,precision_score,0.50,0.602620
4,jaccard_coefficient,precision_score,1.00,0.550218
5,jaccard_coefficient,recall_score,0.10,0.320792
6,jaccard_coefficient,recall_score,0.15,0.324435
7,jaccard_coefficient,recall_score,0.25,0.345982
8,jaccard_coefficient,recall_score,0.50,0.416918
9,jaccard_coefficient,recall_score,1.00,0.510121


## 3

<html><font size=4><B>3. Πρόβλεψη συνδέσμων βάσει similarity-based μετρικών</B></font>    
        <li><B>3(i).</B> Προτείνετε μια τουλάχιστον μέθοδο, για να εξερευνήσουμε και να εκφράσουμε μέσα από τα πακέτα NumPy ή SciPy ή Pandas την συσχέτιση (correlation) μεταξύ των τιμών των μετρικών <B>Jaccard Coefficient</B>, <B>Preferential Attachment</B>, <B>Resource Allocation</B> για τον μειωμένο γράφο $G'$. Σημ.: Δεν χρειάζεται κάποια υλοποίηση, μόνο αναφορά της μεθόδου συσχέτισης και επεξήγηση της επιλογής σας.
     <li><B>3(ii).</B> Εργαστείτε πάνω σε <B><U>αντίγραφο</U></B> του dataframe που κατασκευάσατε στο ερώτημα 1(vα) (unconnected+removables). Δημιουργήστε στήλη (column) με τις ακμές του γράφου και θέστε την ως index του dataframe. Στο dataframe θα προσθέσετε ως στήλες τις τιμές των μετρικών (<B>Jaccard Coefficient</B>, <B>Preferential Attachment</B>, <B>Resource Allocation</B>) για τον μειωμένο γράφο $G'$. Οι τιμές των μετρικών μπορούν να υπολογιστούν με τις συναρτήσεις του πακέτου NetworkX.</li>   </font></html>
    
    |       | link  |Jacc.Coeff.|Pr.Attachment| Resource Allocation|
    |-------|-------|-----------|-------------|--------------------|
    |(1, 6) |  0    |  0.333333 |     32      |    0.700000        |
    |(1, 46)|  0    |  0.200000 |      8      |    0.166667        |

### 3(ii)

In [14]:
df = pd.concat(
    [
        ground_truths, *(
            link_predictor(G_reduced, ground_truths.index)
            for link_predictor in map(link_predictor_wrapper, link_predictors)
        )
    ],
    axis=1
)
df.head()

,is_edge,jaccard_coefficient,preferential_attachment,resource_allocation_index
"(1, 6)",0,0.0,7,0.0
"(1, 46)",0,0.0,1,0.0
"(1, 47)",0,0.0,1,0.0
"(1, 48)",0,0.5,2,0.5
"(1, 32)",0,0.0,6,0.0


### 3(i)

In [15]:
df.corr()

,is_edge,jaccard_coefficient,preferential_attachment,resource_allocation_index
is_edge,1.000000,0.543554,0.011689,0.254448
jaccard_coefficient,0.543554,1.000000,-0.015560,0.630806
preferential_attachment,0.011689,-0.015560,1.000000,0.030255
resource_allocation_index,0.254448,0.630806,0.030255,1.000000


<li><B>3(iiiα).</B> Χρησιμοποιήστε τον Random Forest Classifier και το αντίγραφο dataframe που κατασκευάσατε στο ερώτημα 3(ii), για να προβλέψετε συνδέσεις. Συμπληρώστε στον κώδικα: τα features ($X$), την target variable ($y$) και τα ορίσματα της συνάρτησης $fit$ για το training του μοντέλου.</li>
<li><B>3(iiiβ).</B> Δώστε τις ($A.M$ $mod$ $10$) πρώτες προβλέψεις και την ακρίβεια ($accuracy$) του μοντέλου. </li>
<li><B>3(iiiγ).</B>Υπολογίστε την πιθανότητα να συνδεθούν οι κόμβοι ($1$,$47$) βάσει του μοντέλου σας. </li>  

### 3(iiiα)

In [16]:
# There is no need to rescale when using Decision Trees...
# Also, accuracy score for such an imbalanced dataset provides no proper insight...

from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier

X = df.drop(columns=['is_edge'])
y = df['is_edge']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

grid_search_cv = GridSearchCV(
    RandomForestClassifier(random_state=RANDOM_STATE), {
        'n_estimators': [10, 50, 100],
        'max_depth': [5, 10, 15]
    },
    scoring='f1'
)

grid_search_cv.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(random_state=42),
             param_grid={'max_depth': [5, 10, 15],
                         'n_estimators': [10, 50, 100]},
             scoring='f1')

In [17]:
grid_search_cv.best_score_

0.668194819427321

In [18]:
grid_search_cv.best_params_

{'max_depth': 5, 'n_estimators': 100}

### 3(iiiβ)

In [19]:
preds = grid_search_cv.best_estimator_.predict(X_test)

print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18327
           1       1.00      0.57      0.72        53

    accuracy                           1.00     18380
   macro avg       1.00      0.78      0.86     18380
weighted avg       1.00      1.00      1.00     18380



In [20]:
preds[:K]

array([0, 0, 0, 0, 0, 0, 0, 0, 0])

### 3(iiiγ)

In [21]:
grid_search_cv.best_estimator_.predict_proba(X[X.index == ('1', '47')])[0, 1]

0.00023961118738656601

## 4

<html><font size=4><B>4. Πρόβλεψη συνδέσμων με embedding βασισμένο σε τυχαίους περιπάτους (Random Walks) </B></font><BR><BR>
    <li><B>4(i).</B> Εξάγετε τα χαρακτηριστικά (features) του γράφου $G'$, που προέκυψε από την αφαίρεση των ακμών. Για τον σκοπό αυτό χρησιμοποιήστε τον αλγόριθμο <B>Node2vec</B> <a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3]</a>. Ο αλγόριθμος αντιστοιχίζει τους κόμβους του γράφου μέσω τυχαίων περιπάτων (biased 2nd order random walks) σε ένα χώρο μικρού αριθμού διαστάσεων (low dimensional space) διατηρώντας την μεταξύ τους γειτνίαση.
        <BR>Για την επιλογή των παραμέτρων: 
            <li>p - παράμετρος τυχαίων περιπάτων p που καθορίζει την πιθανότητα “1/p” επιστροφής στον κόμβο προέλευσης (source node), </li>
                <li>q - παράμετρος τυχαίων περιπάτων q που καθορίζει την πιθανότητα  “1/q” μετακίνησης σε κόμβο μακριά από τον κόμβο προέλευσης (source node), </li>
                 <li>dimensions - πλήθος διαστάσεων των Node2Vec embeddings, </li>
                     <li>num_walks - αριθμός περιπάτων από κάθε κόμβο, </li>
                         <li>walk_length - μήκος τυχαίου περιπάτου, </li>
                             <li>window_size - μέγεθος παραθύρου context για τον αλγόριθμο Word2Vec, </li><li>num_iter - αριθμός SGD επαναλήψεων (epochs), </li>
            <li>workers - αριθμός workers για τον Word2Vec,</li>
            χρησιμοποιήστε τις τιμές:   </html>
   
                   

 |   p   |   q   | dimensions|  num_walks  |walk_length |window_size|        workers            |
 |-------|-------|-----------|-------------|------------|-----------|---------------------------|
 |  1.0  |  1.0  |  128      |     10      |    80      |     10    |multiprocessing.cpu_count()|

### 4 (i)

In [22]:
node_2_vec = Node2VecFixed(
    graph=G_reduced,
    dimensions=128,
    walk_length=80,
    num_walks=10,
    p=1.0,
    q=1.0,
    workers=cpu_count(),
)

word_2_vec = node_2_vec.fit(window=10, min_count=1)

Computing transition probabilities: 100%|██████████| 430/430 [00:00<00:00, 14356.74it/s]
Generating walks (CPU: 11): 0it [00:00, ?it/s]1 [00:00<?, ?it/s]]
Generating walks (CPU: 12): 0it [00:00, ?it/s]
Generating walks (CPU: 13): 0it [00:00, ?it/s]
Generating walks (CPU: 15): 0it [00:00, ?it/s]
Generating walks (CPU: 14): 0it [00:00, ?it/s]
Generating walks (CPU: 16): 0it [00:00, ?it/s]
Generating walks (CPU: 9): 100%|██████████| 1/1 [00:02<00:00,  2.93s/it]


<html><font>
    <li><B>4(ii).</B> Εφαρμόστε τον αλγόριθμο Random Forest Classifier, για να προβλέψετε τις συνδέσεις.</li><BR>
    <li><B>4(iii).</B> Υπολογίστε την ακρίβεια ($accuracy$) του μοντέλου. </li><BR></font></html>

### 4(i) & 4(ii) & Bonus Question

In [23]:
def evaluate_rf_n2v(op):
    X = [op(word_2_vec.wv[u], word_2_vec.wv[v]) for u, v in df.index]
    y = df['is_edge']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.8, random_state=RANDOM_STATE)

    rf = RandomForestClassifier(max_depth=20, n_jobs=-1, random_state=RANDOM_STATE)
    rf.fit(X_train, y_train)
    preds = rf.predict(X_test)

    return classification_report(y_test, preds)


def average(x, y):
    return (x + y) / 2


def hadamard(x, y):
    return x * y


def l1(x, y):
    return np.abs(x - y)


def l2(x, y):
    return (x - y)**2


ops = average, hadamard, l1, l2
print(*(f'{op.__name__}\n{evaluate_rf_n2v(op)}' for op in ops), sep='\n')

average
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     73331
           1       0.96      0.49      0.65       189

    accuracy                           1.00     73520
   macro avg       0.98      0.74      0.82     73520
weighted avg       1.00      1.00      1.00     73520

hadamard
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     73331
           1       0.96      0.47      0.63       189

    accuracy                           1.00     73520
   macro avg       0.98      0.73      0.81     73520
weighted avg       1.00      1.00      1.00     73520

l1
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     73331
           1       0.72      0.10      0.17       189

    accuracy                           1.00     73520
   macro avg       0.86      0.55      0.58     73520
weighted avg       1.00      1.00      1.00     73520


**Σχόλιο**

Παρατηρούμε πως τα μέτρα απόστασης αποδίδουν χειρότερα αποτελέσματα, που είναι αναμενόμενο μίας και χάνεται η πληροφορία της τοποθεσίας και κρατείται μόνο η απόσταση μεταξύ των διανυσμάτων αναπαράστασης των κόμβων.

<html><font size=4> <B><U>Ερώτηση bonus (+10/100)</U><B>: </font>Για τους binary operators για learning των edge features με τον αλγόριθμο Node2Vec που αναφέρονται στο <a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3]</a> ((α) Average, (β) Hadamard, (γ) Weighted-L1 και (δ) Weighted-L2) τροποποιήστε τoν κώδικά σας και σημειώστε τις παρατηρήσεις σας ως προς την απόδοση του αλγορίθμου.</font></html>

**Σχόλιο**
Η απάντηση στην ερώτηση bonus βρίσκεται στο προηγούμενο κελί.

<html><font size=4><B><U>Βιβλιογραφία:</U></B><BR></font>
<a href="https://dl.acm.org/doi/10.1145/2487788.2488173">[1]	Jérôme Kunegis. KONECT – The Koblenz Network Collection. In Proc. Int. Conf. on World Wide Web Companion, pages 1343–1350, 2013.</a><BR>
<a href="https://link.springer.com/chapter/10.1007/978-3-540-76298-0_52">[2]	Sören Auer, Christian Bizer, Georgi Kobilarov, Jens Lehmann, Richard Cyganiak, and Zachary Ives. DBpedia: A nucleus for a web of open data. In Proc. Int. Semant. Web Conf., pages 722–735, 2008.<BR>
<a href="https://cs.stanford.edu/~jure/pubs/node2vec-kdd16.pdf">[3] A. Grover, J. Leskovec, Node2vec: Scalable Feature Learning for Networks. ACM KDD, 2016.</a><BR>
 <a href="https://proceedings.neurips.cc/paper/2013/file/9aa42b31882ec039965f3c4923ce901b-Paper.pdf">[4] T.Mikolov, I.Sutskever, K.Chen, G.S.Corrado and J.Dean, Distributed representations of words and phrases and their compositionality. In NIPS, 2013.</a><BR></font></html>